In [ ]:
#TODO: reduce size vocabulary
#TODO: check size embedding
#TODO: tensorboard

In [19]:
import pandas as pd

df = pd.read_csv('data/isear_processed.csv',header=0, index_col=0, names=['text','sentiment'])

In [20]:
df['sentiment'].unique()

array(['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt'],
      dtype=object)

In [21]:
df.head()

,text,sentiment
0,"During the period of falling in love, each tim...",joy
1,When I was involved in a traffic accident.,fear
2,When I was driving home after several days of...,anger
3,When I lost the person who meant the most to me.,sadness
4,The time I knocked a deer down - the sight of ...,disgust


In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
sequences = pad_sequences(sequences, maxlen=100)

In [23]:
import numpy as np

emotion_dict = {}
labels = df['sentiment']

for k in range(len(labels.unique())):
    emotion_dict[labels.unique()[k]] = k
    
encoded_labels = np.array(list(labels.map(lambda x: emotion_dict[x])))

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences, encoded_labels, stratify=encoded_labels)

In [26]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).shuffle(1024).batch(32)

In [36]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout

model = tf.keras.Sequential()

model.add(Embedding(len(tokenizer.word_index) +1, 50))
model.add(LSTM(132, dropout=0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 50)          453200    
_________________________________________________________________
unified_lstm_9 (UnifiedLSTM) (None, 132)               96624     
_________________________________________________________________
dense_13 (Dense)             (None, 32)                4256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 7)                 231       
Total params: 554,311
Trainable params: 554,311
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(X_train, y_train, epochs = 10, validation_data=(X_test,y_test))

Train on 5749 samples, validate on 1917 samples
Epoch 1/10
5749/5749 [==============================] - 20s 4ms/sample - loss: 1.9214 - accuracy: 0.1826 - val_loss: 1.8333 - val_accuracy: 0.3099
Epoch 2/10
5749/5749 [==============================] - 20s 4ms/sample - loss: 1.6058 - accuracy: 0.3910 - val_loss: 1.5251 - val_accuracy: 0.4293
Epoch 3/10
5749/5749 [==============================] - 20s 3ms/sample - loss: 1.2208 - accuracy: 0.5632 - val_loss: 1.4121 - val_accuracy: 0.4956
Epoch 4/10
5749/5749 [==============================] - 20s 3ms/sample - loss: 0.9487 - accuracy: 0.6739 - val_loss: 1.3993 - val_accuracy: 0.5023
Epoch 5/10
5749/5749 [==============================] - 22s 4ms/sample - loss: 0.7846 - accuracy: 0.7393 - val_loss: 1.3972 - val_accuracy: 0.5279
Epoch 6/10
5749/5749 [==============================] - 25s 4ms/sample - loss: 0.6598 - accuracy: 0.7862 - val_loss: 1.5423 - val_accuracy: 0.5143
Epoch 7/10
5749/5749 [==============================] - 23s 4ms/sample